# LangChain Hub

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [ ]:
pip show langchain

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import (create_tool_calling_agent, 
                              AgentExecutor)

In [ ]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [ ]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

In [ ]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [ ]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-4-0125-preview', 
                  temperature = 0,
                  model_kwargs = {'seed':365})

In [ ]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

In [ ]:
chat_prompt_template

In [ ]:
chat_prompt_template.pretty_print()

In [ ]:
ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], 
                   input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
                                                                         langchain_core.messages.human.HumanMessage, 
                                                                         langchain_core.messages.chat.ChatMessage, 
                                                                         langchain_core.messages.system.SystemMessage, 
                                                                         langchain_core.messages.function.FunctionMessage, 
                                                                         langchain_core.messages.tool.ToolMessage]], 
                                'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
                                                                             langchain_core.messages.human.HumanMessage, 
                                                                             langchain_core.messages.chat.ChatMessage, 
                                                                             langchain_core.messages.system.SystemMessage, 
                                                                             langchain_core.messages.function.FunctionMessage, 
                                                                             langchain_core.messages.tool.ToolMessage]]}, 
                   messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], 
                                                                               template='You are a helpful assistant')), 
                             MessagesPlaceholder(variable_name='chat_history', optional=True), 
                             HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], 
                                                                              template='{input}')), 
                             MessagesPlaceholder(variable_name='agent_scratchpad')])

In [ ]:
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [ ]:
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True, 
                               return_intermediate_steps=True)

In [ ]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})